![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook - Prompt Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: [Saving your work in Prompt Lab as a notebook](/docs).

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Creating an access token from the IBM Cloud personal API key
* Defining a Python class for calling the WML Foundation Model inferencing API
* Using the class to generate output from a provided text input

# Setup

## Inferencing class
This cell defines a class that makes a REST API call to the watsonx Foundation Model
inferencing API that we will use to generate output from the provided input.
The class takes the access token created in the previous step, and uses it to
make a REST API call with input, model id and model parameters. The response
from the API call is returned as the cell output.

In [1]:
import requests

class Prompt:
    def __init__(self, access_token, project_id):
        self.access_token = access_token
        self.project_id = project_id

    def generate(self, input, model_id, parameters):
        wml_url = "https://us-south.ml.cloud.ibm.com/ml/v1-beta/generation/text?version=2023-05-28"
        Headers = {
            "Authorization": "Bearer " + self.access_token,
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        data = {
            "model_id": model_id,
            "input": input,
            "parameters": parameters,
            "project_id": self.project_id
        }
        response = requests.post(wml_url, json=data, headers=Headers)
        if response.status_code == 200:
            return response.json()["results"][0]["generated_text"]
        else:
            return response.text

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [2]:
from ibm_cloud_sdk_core import IAMTokenManager
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator
import os, getpass

access_token = IAMTokenManager(
    apikey = getpass.getpass("Please enter your api key (hit enter): "),
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

Please enter your api key (hit enter): ········


# Inferencing
This cell demonstrated how we can use the defined class as well as the
created access token to pair it with model id, parameters and input string
to obtain the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [3]:
model_id = "ibm/mpt-7b-instruct2"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [4]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 200,
    "repetition_penalty": 1
}

## Defining the project id
The API requires project id that provides the context for the call. We will obtain
the id from the project in which this notebook runs:

In [5]:
import os

project_id = os.environ["PROJECT_ID"]

## Defining the inferencing input
Foundation model inferencing API accepts a natural language input that it will use
to provide the natural language response. The API is sensitive to formatting. Input
structure, presence of training steps (one-shot, two-shot learning etc.), as well
as phrasing all influence the final response and belongs to the emerging discipline of
Prompt Engineering.

Let us provide the input we got from the Prompt Lab:

In [6]:
prompt_input = """Summarize the article

Input:
To the dedicated and hardworking librarians of America:

In any democracy, the free exchange of ideas is an important part of making sure that citizens are informed, engaged and feel like their perspectives matter.

It’s so important, in fact, that here in America, the First Amendment of our Constitution states that freedom begins with our capacity to share and access ideas — even, and maybe especially, the ones we disagree with.

More often than not, someone decides to write those ideas down in a book.

Books have always shaped how I experience the world. Writers like Mark Twain and Toni Morrison, Walt Whitman and James Baldwin taught me something essential about our country’s character. Reading about people whose lives were very different from mine showed me how to step into someone else’s shoes. And the simple act of writing helped me develop my own identity — all of which would prove vital as a citizen, as a community organizer, and as president.

Today, some of the books that shaped my life — and the lives of so many others — are being challenged by people who disagree with certain ideas or perspectives. It’s no coincidence that these “banned books” are often written by or feature people of color, indigenous people, and members of the LGBTQ+ community — though there have also been unfortunate instances in which books by conservative authors or books containing “triggering” words or scenes have been targets for removal. Either way, the impulse seems to be to silence, rather than engage, rebut, learn from or seek to understand views that don’t fit our own.

I believe such an approach is profoundly misguided, and contrary to what has made this country great. As I’ve said before, not only is it important for young people from all walks of life to see themselves represented in the pages of books, but it’s also important for all of us to engage with different ideas and points of view.

It’s also important to understand that the world is watching. If America — a nation built on freedom of expression — allows certain voices and ideas to be silenced, why should other countries go out of their way to protect them? Ironically, it is Christian and other religious texts — the sacred texts that some calling for book bannings in this country claim to want to defend — that have often been the first target of censorship and book banning efforts in authoritarian countries.

Nobody understands that more than you, our nation’s librarians. In a very real sense, you’re on the front lines — fighting every day to make the widest possible range of viewpoints, opinions, and ideas available to everyone. Your dedication and professional expertise allow us to freely read and consider information and ideas, and decide for ourselves which ones we agree with.

That’s why I want to take a moment to thank all of you for the work you do every day — work that is helping us understand each other and embrace our shared humanity.

And it’s not just about books. You also provide spaces where people can come together, share ideas, participate in community programs, and access essential civic and educational resources. Together, you help people become informed and active citizens, capable of making this country what they want it to be.

And you do it all in a harsh political climate where, all too often, you’re attacked by people who either cannot or will not understand the vital — and uniquely American — role you play in the life of our nation.

So whether you just started working at a school or public library, or you’ve been there your entire career, Michelle and I want to thank you for your unwavering commitment to the freedom to read. All of us owe you a debt of gratitude for making sure readers across the country have access to a wide range of books, and all the ideas they contain.

Finally, to every citizen reading this, I hope you’ll join me in reminding anyone who will listen — and even some people you think might not — that the free, robust exchange of ideas has always been at the heart of American democracy. Together, we can make that true for generations to come.

Output:
Librarians are at the front lines of defending our Constitutional right to free speech.  They ensure that libraries carry books that cover a broad spectrum of thought, even if we may not agree with the thoughts expressed in some of the books.  They should be thanked for the tireless work they do.

Input:
To the dedicated and hardworking librarians of America:

In any democracy, the free exchange of ideas is an important part of making sure that citizens are informed, engaged and feel like their perspectives matter.

It’s so important, in fact, that here in America, the First Amendment of our Constitution states that freedom begins with our capacity to share and access ideas — even, and maybe especially, the ones we disagree with.

More often than not, someone decides to write those ideas down in a book.

Books have always shaped how I experience the world. Writers like Mark Twain and Toni Morrison, Walt Whitman and James Baldwin taught me something essential about our country’s character. Reading about people whose lives were very different from mine showed me how to step into someone else’s shoes. And the simple act of writing helped me develop my own identity — all of which would prove vital as a citizen, as a community organizer, and as president.

Today, some of the books that shaped my life — and the lives of so many others — are being challenged by people who disagree with certain ideas or perspectives. It’s no coincidence that these “banned books” are often written by or feature people of color, indigenous people, and members of the LGBTQ+ community — though there have also been unfortunate instances in which books by conservative authors or books containing “triggering” words or scenes have been targets for removal. Either way, the impulse seems to be to silence, rather than engage, rebut, learn from or seek to understand views that don’t fit our own.

I believe such an approach is profoundly misguided, and contrary to what has made this country great. As I’ve said before, not only is it important for young people from all walks of life to see themselves represented in the pages of books, but it’s also important for all of us to engage with different ideas and points of view.

It’s also important to understand that the world is watching. If America — a nation built on freedom of expression — allows certain voices and ideas to be silenced, why should other countries go out of their way to protect them? Ironically, it is Christian and other religious texts — the sacred texts that some calling for book bannings in this country claim to want to defend — that have often been the first target of censorship and book banning efforts in authoritarian countries.

Nobody understands that more than you, our nation’s librarians. In a very real sense, you’re on the front lines — fighting every day to make the widest possible range of viewpoints, opinions, and ideas available to everyone. Your dedication and professional expertise allow us to freely read and consider information and ideas, and decide for ourselves which ones we agree with.

That’s why I want to take a moment to thank all of you for the work you do every day — work that is helping us understand each other and embrace our shared humanity.

And it’s not just about books. You also provide spaces where people can come together, share ideas, participate in community programs, and access essential civic and educational resources. Together, you help people become informed and active citizens, capable of making this country what they want it to be.

And you do it all in a harsh political climate where, all too often, you’re attacked by people who either cannot or will not understand the vital — and uniquely American — role you play in the life of our nation.

So whether you just started working at a school or public library, or you’ve been there your entire career, Michelle and I want to thank you for your unwavering commitment to the freedom to read. All of us owe you a debt of gratitude for making sure readers across the country have access to a wide range of books, and all the ideas they contain.

Finally, to every citizen reading this, I hope you’ll join me in reminding anyone who will listen — and even some people you think might not — that the free, robust exchange of ideas has always been at the heart of American democracy. Together, we can make that true for generations to come.

Output:
"""

## Execution
Let us now use the defined variables to create an instance of the class
we defined previously and get the response from the selected foundation model:

In [7]:
prompt = Prompt(access_token, project_id)

prompt.generate(prompt_input, model_id, parameters)

'Librarians are at the front lines of defending our Constitutional right to free speech.  They ensure that libraries carry books that cover a broad spectrum of thought, even if we may not agree with the thoughts expressed in some of the books.  They should be thanked for the tireless work they do.'

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing API to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF">License Terms</a>  